<a href="https://colab.research.google.com/github/Khoawawa/Transformer/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"khoawawa","key":"0be104a1b4ed72031ed77f0d0cf80cd9"}'}

In [2]:
import os
import shutil

# Create the .kaggle directory
os.makedirs("/root/.kaggle", exist_ok=True)

# Move the kaggle.json file
shutil.move('kaggle.json', "/root/.kaggle/kaggle.json")

# Set permissions to avoid security issues
os.chmod("/root/.kaggle/kaggle.json", 0o600)

In [3]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticate
api = KaggleApi()
api.authenticate()

# Download the dataset
dataset_name = "dhruvildave/en-fr-translation-dataset"
api.dataset_download_files(dataset_name, path="/content/", unzip=True)

Dataset URL: https://www.kaggle.com/datasets/dhruvildave/en-fr-translation-dataset


KeyboardInterrupt: 

In [4]:
dataset_path = "/content/en-fr.csv"

In [5]:
!git clone https://github.com/Khoawawa/Transformer.git

fatal: destination path 'Transformer' already exists and is not an empty directory.


In [18]:
import torch
from torch.utils.data import DataLoader
from Transformer.transformer import Transformer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from transformers import AutoTokenizer

In [19]:
df = pd.read_csv(dataset_path,nrows=80000)
df = df.dropna()
df.head(5)

,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English


In [20]:
df = df.apply(lambda x: x.astype(str).str.lower())
df.head(5)

,en,fr
0,changing lives | changing society | how it wor...,il a transformé notre vie | il a transformé la...
1,site map,plan du site
2,feedback,rétroaction
3,credits,crédits
4,français,english


In [21]:
def remove_punctuation(text):
  return ''.join([char for char in text if char not in string.punctuation])

df.apply(lambda x: remove_punctuation(x))
df.head(5)

,en,fr
0,changing lives | changing society | how it wor...,il a transformé notre vie | il a transformé la...
1,site map,plan du site
2,feedback,rétroaction
3,credits,crédits
4,français,english


In [22]:
df = df.map(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x)
df.head(5)

,en,fr
0,changing lives | changing society | how it wor...,il a transformé notre vie | il a transformé la...
1,site map,plan du site
2,feedback,rétroaction
3,credits,crédits
4,français,english


In [23]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [24]:
df["en_tokens"] = df["en"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True, return_tensors="pt"))
df["fr_tokens"] = df["fr"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True, return_tensors="pt"))

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

save_path = "/content/drive/MyDrive/tokenized_data.pkl"  # Adjust path as needed

with open(save_path, "wb") as f:
    pickle.dump(df, f)

print(f"Tokenized dataset saved to {save_path}")